In [5]:
# The code was removed by DSX for sharing.

[Row(PassengerId=u'1', Survived=u'0', Pclass=u'3', Name=u'Braund, Mr. Owen Harris', Sex=u'male', Age=u'22', SibSp=u'1', Parch=u'0', Ticket=u'A/5 21171', Fare=u'7.25', Cabin=None, Embarked=u'S'),
 Row(PassengerId=u'2', Survived=u'1', Pclass=u'1', Name=u'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex=u'female', Age=u'38', SibSp=u'1', Parch=u'0', Ticket=u'PC 17599', Fare=u'71.2833', Cabin=u'C85', Embarked=u'C'),
 Row(PassengerId=u'3', Survived=u'1', Pclass=u'3', Name=u'Heikkinen, Miss. Laina', Sex=u'female', Age=u'26', SibSp=u'0', Parch=u'0', Ticket=u'STON/O2. 3101282', Fare=u'7.925', Cabin=None, Embarked=u'S'),
 Row(PassengerId=u'4', Survived=u'1', Pclass=u'1', Name=u'Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex=u'female', Age=u'35', SibSp=u'1', Parch=u'0', Ticket=u'113803', Fare=u'53.1', Cabin=u'C123', Embarked=u'S'),
 Row(PassengerId=u'5', Survived=u'0', Pclass=u'3', Name=u'Allen, Mr. William Henry', Sex=u'male', Age=u'35', SibSp=u'0', Parch=u'0', Ticket=u'373450', Far

In [6]:
trainingDF.count()

891

In [7]:
import pandas as pd
trainingDF = (trainingDF.withColumn('Parch',trainingDF['Parch'].cast('double'))
                        .withColumn('Survived',trainingDF['Survived'].cast('double'))
                        .withColumn('SibSp',trainingDF['SibSp'].cast('double'))
                        .withColumn('Age',trainingDF['Age'].cast('double'))
                        .withColumn('Fare',trainingDF['Fare'].cast('double')))

In [11]:
trainingDF = trainingDF.fillna({'Sex': 'Null', 'Ticket': 'Null','Age': trainingDF.groupBy().mean('Age').first()[0], 'Fare':trainingDF.groupBy().mean('Fare').first()[0], 'Survived':0, 'SibSp': 0, 'Parch': 0, 'Pclass': 'Null','Embarked': 'Null'})

In [12]:
trainingDF = trainingDF.drop('Cabin', 'PassengerId','Name')

In [14]:
trainingDF = trainingDF.dropna()

In [15]:
trainingDF.count()

891

In [16]:
###One-Hot Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ["Pclass", "Sex", "Ticket", "Embarked"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer,encoder]

In [17]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "Survived", outputCol = "label")
stages += [label_stringIdx]

In [18]:
# Transform all features into a vector using VectorAssembler
numericCols = ["Age", "SibSp","Parch", "Fare"]
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [20]:
cols = trainingDF.columns
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(trainingDF)
dataset = pipelineModel.transform(trainingDF)

In [21]:
# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)
dataset.toPandas()

,label,features,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,3,male,22.000000,1,0,A/5 21171,7.2500,S
1,1,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,1,female,38.000000,1,0,PC 17599,71.2833,C
2,1,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,3,female,26.000000,0,0,STON/O2. 3101282,7.9250,S
3,1,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,1,female,35.000000,1,0,113803,53.1000,S
4,0,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,3,male,35.000000,0,0,373450,8.0500,S
5,0,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,3,male,29.699118,0,0,330877,8.4583,Q
6,0,"(0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,1,male,54.000000,0,0,17463,51.8625,S
7,0,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,3,male,2.000000,3,1,349909,21.0750,S
8,1,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,3,female,27.000000,0,2,347742,11.1333,S
9,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,2,female,14.000000,1,0,237736,30.0708,C


In [22]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.75, 0.25], seed = 100)

In [24]:
print trainingData.count()
print testData.count()

657
234


In [25]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)
predictions.printSchema()

# View model's predictions and probabilities of each prediction class
# You can select any columns in the above schema to view as well. For example's sake we will choose age & sex
selected = predictions.select("label", "prediction", "probability","Age", "Sex")
selected.toPandas()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- Survived: double (nullable = false)
 |-- Pclass: string (nullable = false)
 |-- Sex: string (nullable = false)
 |-- Age: double (nullable = false)
 |-- SibSp: double (nullable = false)
 |-- Parch: double (nullable = false)
 |-- Ticket: string (nullable = false)
 |-- Fare: double (nullable = false)
 |-- Embarked: string (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



,label,prediction,probability,Age,Sex
0,0,0,"[0.999981670013, 1.83299867073e-05]",29.699118,male
1,0,1,"[0.229418030563, 0.770581969437]",29.699118,male
2,0,0,"[0.999975785539, 2.42144611247e-05]",1.000000,male
3,0,0,"[0.999976528511, 2.34714889607e-05]",2.000000,male
4,0,0,"[0.99999775835, 2.24164952953e-06]",40.000000,male
5,0,0,"[0.999978494472, 2.15055277639e-05]",11.000000,male
6,0,0,"[0.999980413923, 1.9586077438e-05]",14.000000,male
7,0,0,"[0.999980710159, 1.92898414733e-05]",4.000000,male
8,0,0,"[0.725264179845, 0.274735820155]",36.000000,male
9,0,0,"[0.798325613086, 0.201674386914]",49.000000,male


In [26]:


from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

#evaluate the LogisticRegression Model predictions
evaluator.evaluate(predictions)

0.8198279087167974

In [27]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)

In [28]:
predictions = cvModel.transform(testData)
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.8539094650205764

In [66]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)

# Train model with Training Data
dtModel = dt.fit(trainingData)

In [67]:
# Make predictions on test data using the Transformer.transform() method.
predictions = dtModel.transform(testData)
selected = predictions.select("label", "prediction", "probability", "Age","Sex")
selected.show()

+-----+----------+--------------------+-------------+----+
|label|prediction|         probability|          Age| Sex|
+-----+----------+--------------------+-------------+----+
|  0.0|       0.0|[0.89329268292682...|29.6991176471|male|
|  0.0|       0.0|[0.89329268292682...|29.6991176471|male|
|  0.0|       0.0|         [0.75,0.25]|          1.0|male|
|  0.0|       0.0|         [0.75,0.25]|          2.0|male|
|  0.0|       0.0|[0.89329268292682...|         40.0|male|
|  0.0|       0.0|[0.89329268292682...|         11.0|male|
|  0.0|       0.0|[0.89329268292682...|         14.0|male|
|  0.0|       0.0|[0.89329268292682...|          4.0|male|
|  0.0|       0.0|[0.89329268292682...|         36.0|male|
|  0.0|       0.0|[0.89329268292682...|         49.0|male|
|  0.0|       0.0|[0.89329268292682...|         36.0|male|
|  0.0|       0.0|[0.89329268292682...|29.6991176471|male|
|  0.0|       0.0|[0.89329268292682...|         18.0|male|
|  0.0|       0.0|[0.89329268292682...|         27.0|mal

In [68]:
#evaluate decision tree classifier model

evaluator.evaluate(predictions)

0.793939393939394

In [69]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1,2,6,10])
             .addGrid(dt.maxBins, [20,40,80])
             .build())
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)


In [70]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.8292929292929293

In [82]:
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)


predictions = rfModel.transform(testData)

In [83]:

# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "Age","Sex")
selected.show()

+-----+----------+--------------------+-------------+----+
|label|prediction|         probability|          Age| Sex|
+-----+----------+--------------------+-------------+----+
|  0.0|       0.0|[0.67911771158113...|29.6991176471|male|
|  0.0|       0.0|[0.68688287439747...|29.6991176471|male|
|  0.0|       0.0|[0.61827801566730...|          1.0|male|
|  0.0|       0.0|[0.61827801566730...|          2.0|male|
|  0.0|       0.0|[0.67244257151202...|         40.0|male|
|  0.0|       0.0|[0.67568409193471...|         11.0|male|
|  0.0|       0.0|[0.67568409193471...|         14.0|male|
|  0.0|       0.0|[0.72489366119374...|          4.0|male|
|  0.0|       0.0|[0.74937366144782...|         36.0|male|
|  0.0|       0.0|[0.73827457534874...|         49.0|male|
|  0.0|       0.0|[0.67244257151202...|         36.0|male|
|  0.0|       0.0|[0.67244257151202...|29.6991176471|male|
|  0.0|       0.0|[0.67244257151202...|         18.0|male|
|  0.0|       0.0|[0.67142542712931...|         27.0|mal

In [84]:
evaluator.evaluate(predictions)


0.6730639730639731

In [85]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(trainingData)

In [87]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [88]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.694949494949495

In [98]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

features_size = trainingData.select("features").first()[0].size
layers = [features_size, 10, 2]
# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(trainingData)


In [99]:
predictions = model.transform(trainingData)
selected = predictions.select("label", "prediction", "Age","Sex")
selected.show()

+-----+----------+-------------+----+
|label|prediction|          Age| Sex|
+-----+----------+-------------+----+
|  0.0|       0.0|          4.0|male|
|  0.0|       0.0|         39.0|male|
|  0.0|       0.0|29.6991176471|male|
|  0.0|       0.0|29.6991176471|male|
|  0.0|       0.0|29.6991176471|male|
|  0.0|       0.0|         28.0|male|
|  0.0|       0.0|          7.0|male|
|  0.0|       0.0|         14.0|male|
|  0.0|       0.0|         16.0|male|
|  0.0|       0.0|          4.0|male|
|  0.0|       0.0|         10.0|male|
|  0.0|       0.0|          1.0|male|
|  0.0|       0.0|          9.0|male|
|  0.0|       0.0|          2.0|male|
|  0.0|       0.0|          7.0|male|
|  0.0|       0.0|          8.0|male|
|  0.0|       0.0|         19.0|male|
|  0.0|       0.0|          2.0|male|
|  0.0|       0.0|         16.0|male|
|  0.0|       0.0|          9.0|male|
+-----+----------+-------------+----+
only showing top 20 rows



In [100]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
evaluator.evaluate(predictions)


0.7841295401681875